In [ ]:
from kedro.config import OmegaConfigLoader

loader = OmegaConfigLoader(conf_source="conf/base")
params = loader["parameters"]
print(params["Control_date"]["centaline_estates"])  # Should output "2025-04-09"


In [ ]:
# Change CSV to Parquet

import pandas as pd

def csv_to_parquet(csv_file_path, parquet_file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)
        
        # Write the DataFrame to a Parquet file
        df.to_parquet(parquet_file_path, index=False)
        
        print(f"Successfully converted {csv_file_path} to {parquet_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
csv_file_path = '2025-04-02_centanet_estates.csv'
parquet_file_path = 'estate_listings.parquet'

# Ensure pandas is installed
# pip install pandas

csv_to_parquet(csv_file_path, parquet_file_path)


Successfully converted 2025-04-02_centanet_estates_scraped.csv to estate_details_raw.parquet


In [13]:
import pandas as pd

parquet_file_path = 'data/02_intermediate/centaline_estate_lv_1.parquet'
pq_file = pd.read_parquet(parquet_file_path)

pq_file[pq_file['Name']=='']

,Name,Address,Blocks,Units,UnitRate,MoM,ForSale,ForRent,Link,Region,District,Subdistrict,Code,LastScraped


In [1]:
import pandas as pd

pq_1_path = 'data/02_intermediate/centaline_estate_lv_1.parquet'
#pq_2_path = 'data/03_intermediate/centaline_estate_lv_2.parquet'

pq_2_path = 'data/03_primary/midland_res_trans_lv_2.parquet'
pq_file = pd.read_parquet(pq_2_path)

pq_file

,id_trans,region_id_trans,region_name_trans,subregion_id_trans,subregion,district_id_trans,district,sm_district_id_trans,sm_district,combined_district_id_trans,...,market_stat_monthly_10_date,market_stat_monthly_10_avg_net_ft_price,market_stat_monthly_11_date,market_stat_monthly_11_avg_net_ft_price,index_component_estate_net_ft_price,index_component_estate_net_ft_price_chg,parent_estate_id,parent_estate_name,price_per_sqft,price_variance
0,I20250301813,10,Hong Kong Island,1001,Central And Western,100103,Mid-Levels West,10010001,Kennedy Town,100103-10010001,...,2025-01-31T16:00:00.000Z,19933.0,2025-02-28T16:00:00.000Z,19478.0,19449.0,-0.025702,,,48.834628,-20122.165372
1,I20250301813,10,Hong Kong Island,1001,Central And Western,100103,Mid-Levels West,10010001,Kennedy Town,100103-10010001,...,2025-01-31T16:00:00.000Z,19933.0,2025-02-28T16:00:00.000Z,19478.0,19449.0,-0.025702,,,48.834628,-20122.165372
2,I20250302773,10,Hong Kong Island,1001,Central And Western,100103,Mid-Levels West,10010005,Mid-Levels West,100103-10010005,...,2025-01-31T16:00:00.000Z,15753.0,2025-02-28T16:00:00.000Z,15753.0,NaN,NaN,,,49.315068,-15703.684932
3,I20250400203,10,Hong Kong Island,1001,Central And Western,100103,Mid-Levels West,10010006,Mid-Levels Central,100103-10010006,...,2025-01-31T16:00:00.000Z,16334.0,2025-02-28T16:00:00.000Z,16334.0,NaN,NaN,,,18372.703412,611.703412
4,I20250301892,10,Hong Kong Island,1001,Central And Western,100103,Mid-Levels West,10010005,Mid-Levels West,100103-10010005,...,2025-01-31T16:00:00.000Z,22373.0,2025-02-28T16:00:00.000Z,22373.0,NaN,NaN,,,18118.811881,-0.188119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234138,NO2021050521050501840081,30,New Territories,3018,Islands,301803,Tung Chung,30180002,Tung Chung,301803-30180002,...,2025-01-31T16:00:00.000Z,9990.0,2025-02-28T16:00:00.000Z,11083.0,11052.0,-3.711448,,,12808.022923,1460.022923
234139,NO2021050421050401950028,30,New Territories,3018,Islands,301803,Tung Chung,30180002,Tung Chung,301803-30180002,...,2025-01-31T16:00:00.000Z,9990.0,2025-02-28T16:00:00.000Z,11083.0,11052.0,-3.711448,,,15463.917526,4115.917526
234140,NO2021050321050301110058,30,New Territories,3018,Islands,301803,Tung Chung,30180002,Tung Chung,301803-30180002,...,2025-01-31T16:00:00.000Z,10644.0,2025-02-28T16:00:00.000Z,11353.0,11127.0,-0.233121,,,13141.524106,1833.524106
234141,NO2021043021043002190055,30,New Territories,3018,Islands,301803,Tung Chung,30180002,Tung Chung,301803-30180002,...,2025-01-31T16:00:00.000Z,10644.0,2025-02-28T16:00:00.000Z,11353.0,11127.0,-0.233121,,,12954.898911,1646.898911


In [16]:
# Grouping by 'Code' and calculating row count
row_count_by_code_1 = pq_file.groupby(['Subdistrict', 'Code']).size().reset_index(name='Row Count')
row_count_by_code_2 = pq_file.groupby(['Code']).size().reset_index(name='Row Count')


# Displaying the result HMA151
print(row_count_by_code_1[row_count_by_code_1['Code']=='HMA111'])
print(row_count_by_code_2[row_count_by_code_2['Code']=='HMA111'])



     Subdistrict    Code  Row Count
39  Kennedy Town  HMA111        199
       Code  Row Count
100  HMA111        199


In [2]:
import pandas as pd

pq_path = 'data/02_intermediate/centanet_oir_buildings.parquet'

pq_file = pd.read_parquet(pq_path)

pq_file.to_csv('centanet_oir_buildings.csv', index=False)

In [15]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import logging
import os
from typing import Dict, List, Optional, Union, Any

def scrape_midland_buildings(
    csv_path: str, 
    output_path: str = "midland_buildings.csv",
    request_delay: float = 0.5,
    max_retries: int = 3,
    log_level: int = logging.INFO,
    save_incremental: bool = True,
    incremental_save_frequency: int = 5,
    resume_from_existing: bool = True
) -> pd.DataFrame:
    """
    Scrape building information from Midland ICI GraphQL API for all districts
    and property types (Industrial, Office, Shop).
    
    Args:
        csv_path (str): Path to the CSV file containing district IDs
        output_path (str): Path where the output CSV will be saved
        request_delay (float): Delay between requests in seconds to avoid rate limiting
        max_retries (int): Maximum number of retries for failed requests
        log_level (int): Logging level (e.g., logging.INFO, logging.DEBUG)
        save_incremental (bool): Whether to save incremental results
        incremental_save_frequency (int): How often to save incremental results
        resume_from_existing (bool): Whether to resume from existing output file
        
    Returns:
        pd.DataFrame: DataFrame containing all scraped building information
    """
    # Configure logging
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger("midland_scraper")
    
    # Define property types
    property_types = {
        "mr_ind": "Industrial",
        "mr_comm": "Office",
        "mr_shop": "Shop"
    }
    
    # Define the GraphQL endpoint
    url = "https://service.midlandici.com/building/graphql"
    
    # Define the headers
    headers = {
        "Content-Type": "application/json",
        "Accept": "*/*",
        "Origin": "https://www.midlandici.com.hk",
        "Referer": "https://www.midlandici.com.hk/",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
    }
    
    # Read the CSV file with district IDs
    try:
        districts_df = pd.read_csv(csv_path)
        logger.info(f"Successfully loaded {len(districts_df)} districts from {csv_path}")
    except Exception as e:
        logger.error(f"Failed to load CSV file {csv_path}: {str(e)}")
        return pd.DataFrame()
    
    # Filter out the "All Districts" row (ID=0)
    districts_df = districts_df[districts_df['ID'] != 0]
    logger.info(f"Filtered to {len(districts_df)} districts (excluding 'All Districts')")
    
    # Check if we should resume from existing file
    already_scraped = set()
    existing_buildings_df = pd.DataFrame()
    
    if resume_from_existing and os.path.exists(output_path):
        try:
            existing_buildings_df = pd.read_csv(output_path)
            logger.info(f"Found existing output file with {len(existing_buildings_df)} buildings")
            
            # Create a set of already scraped district-property type combinations
            if 'district_id' in existing_buildings_df.columns and 'property_type_code' in existing_buildings_df.columns:
                already_scraped = set(
                    existing_buildings_df[['district_id', 'property_type_code']].drop_duplicates().apply(
                        lambda x: f"{x['district_id']}_{x['property_type_code']}", axis=1
                    )
                )
                logger.info(f"Found {len(already_scraped)} already scraped district-property type combinations")
        except Exception as e:
            logger.warning(f"Could not read existing output file: {str(e)}. Starting from scratch.")
    
    # Initialize an empty list to store all building data
    all_buildings = []
    
    # Keep track of how many combinations we've processed for incremental saving
    processed_count = 0
    
    # Calculate total iterations for tqdm
    total_iterations = len(districts_df) * len(property_types)
    
    # Create a progress bar
    with tqdm(
        total=total_iterations,
        desc="Scraping progress",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}] {postfix}"
    ) as pbar:
        for _, district in districts_df.iterrows():
            district_id = district['ID']
            district_name_en = district['Name_EN']
            district_name_cn = district['Name_CN']
            
            for sbu, property_type in property_types.items():
                # Update progress bar suffix
                pbar.set_postfix({
                    'District': district_name_en,
                    'Type': property_type
                })
                pbar.refresh()  # Force immediate update

                # Check if this combination has already been scraped
                combo_key = f"{district_id}_{sbu}"
                if combo_key in already_scraped:
                    #logger.info(f"Skipping already scraped {property_type} in {district_name_en}")
                    pbar.update(1)
                    continue
                
                # Update progress bar description
                #pbar.set_description(f"Scraping {property_type} in {district_name_en}")
                
                # Define the GraphQL query and variables
                payload = {
                    "query": """
                        query ($districtId: ID, $query: String, $sbu: String) {
                          buildings(districtId: $districtId, nameSearch: $query, sbu: $sbu) {
                            sbu
                            id
                            nameEn
                            nameZh
                            addressEn
                            addressZh
                            __typename
                          }
                        }
                    """,
                    "variables": {
                        "sbu": sbu,
                        "districtId": district_id,
                        "query": ""
                    }
                }
                
                # Implement retry mechanism
                retries = 0
                success = False
                
                while retries < max_retries and not success:
                    try:
                        # Make the POST request
                        response = requests.post(url, json=payload, headers=headers)
                        
                        # Check if the request was successful
                        if response.status_code == 200:
                            data = response.json()
                            
                            # Check if there are buildings in the response
                            if 'data' in data and 'buildings' in data['data']:
                                buildings = data['data']['buildings']
                                
                                if buildings:
                                    #logger.info(f"Found {len(buildings)} {property_type} buildings in {district_name_en}")
                                    
                                    # Add district and property type information to each building
                                    for building in buildings:
                                        building['district_id'] = district_id
                                        building['district_name_en'] = district_name_en
                                        building['district_name_cn'] = district_name_cn
                                        building['property_type'] = property_type
                                        building['property_type_code'] = sbu
                                        
                                        # Add to the list of all buildings
                                        all_buildings.append(building)
                                #else:
                                    #logger.info(f"No {property_type} buildings found in {district_name_en}")
                                
                                success = True
                            else:
                                logger.warning(f"Unexpected response structure for {district_name_en}, {property_type}")
                                retries += 1
                        else:
                            logger.warning(f"Request failed for {district_name_en}, {property_type} with status code {response.status_code}")
                            retries += 1
                    
                    except Exception as e:
                        logger.error(f"Error occurred for {district_name_en}, {property_type}: {str(e)}")
                        retries += 1
                    
                    # If this isn't the last retry and we haven't succeeded, wait before retrying
                    if retries < max_retries and not success:
                        time.sleep(request_delay * 2)  # Longer delay for retries
                
                # Update processed count
                processed_count += 1
                
                # Save incremental results if needed
                if save_incremental and processed_count % incremental_save_frequency == 0:
                    _save_incremental_results(
                        all_buildings, existing_buildings_df, output_path, logger
                    )
                
                # Update progress bar
                pbar.update(1)
                
                # Add a small delay to avoid rate limiting
                time.sleep(request_delay)
    
    # Convert the list of buildings to a DataFrame
    buildings_df = _process_and_save_results(
        all_buildings, existing_buildings_df, output_path, logger
    )
    
    return buildings_df

def _process_and_save_results(
    new_buildings: List[Dict[str, Any]], 
    existing_buildings_df: pd.DataFrame,
    output_path: str,
    logger: logging.Logger
) -> pd.DataFrame:
    """
    Process new buildings data, combine with existing data if any, and save to CSV.
    
    Args:
        new_buildings: List of new building dictionaries
        existing_buildings_df: DataFrame of existing buildings (can be empty)
        output_path: Path to save the final CSV
        logger: Logger instance
        
    Returns:
        Combined DataFrame of all buildings
    """
    if not new_buildings and existing_buildings_df.empty:
        logger.warning("No buildings found in any district for any property type")
        return pd.DataFrame()
    
    # Convert new buildings to DataFrame
    if new_buildings:
        new_buildings_df = pd.DataFrame(new_buildings)
        
        # Add suffix information for clarity
        new_buildings_df['district_property_type'] = new_buildings_df.apply(
            lambda row: f"{row['district_name_en']}_{row['property_type']}", axis=1
        )
    else:
        new_buildings_df = pd.DataFrame()
    
    # Combine with existing data if there is any
    if not existing_buildings_df.empty and not new_buildings_df.empty:
        # Ensure 'district_property_type' exists in existing data
        if 'district_property_type' not in existing_buildings_df.columns:
            existing_buildings_df['district_property_type'] = existing_buildings_df.apply(
                lambda row: f"{row['district_name_en']}_{row['property_type']}" 
                if 'district_name_en' in existing_buildings_df.columns and 'property_type' in existing_buildings_df.columns 
                else "", axis=1
            )
        
        # Combine DataFrames
        combined_df = pd.concat([existing_buildings_df, new_buildings_df], ignore_index=True)
        
        # Remove duplicates based on building ID and property type
        if 'id' in combined_df.columns and 'property_type_code' in combined_df.columns:
            combined_df = combined_df.drop_duplicates(subset=['id', 'property_type_code'])
    elif not new_buildings_df.empty:
        combined_df = new_buildings_df
    else:
        combined_df = existing_buildings_df
    
    # Save to CSV
    #try:
    #    combined_df.to_csv(output_path, index=False)
        #logger.info(f"Successfully saved {len(combined_df)} buildings to {output_path}")
    #except Exception as e:
    #    logger.error(f"Failed to save CSV file: {str(e)}")
    
    return combined_df

def _save_incremental_results(
    new_buildings: List[Dict[str, Any]], 
    existing_buildings_df: pd.DataFrame,
    output_path: str,
    logger: logging.Logger
) -> None:
    """
    Save incremental results to avoid data loss if the process is interrupted.
    
    Args:
        new_buildings: List of new building dictionaries
        existing_buildings_df: DataFrame of existing buildings (can be empty)
        output_path: Path to save the incremental CSV
        logger: Logger instance
    """
    if not new_buildings:
        logger.debug("No new buildings to save incrementally")
        return
    
    try:
        # Process and save results
        _process_and_save_results(
            new_buildings, existing_buildings_df, output_path, logger
        )
        #logger.info(f"Saved incremental results with {len(new_buildings)} new buildings")
    except Exception as e:
        logger.error(f"Failed to save incremental results: {str(e)}")

# Example usage
if __name__ == "__main__":
    # Replace with your actual CSV path
    csv_path = "data/01_raw/midland_ici_area_code.csv"
    output_path = "midland_buildings.csv"
    
    # Run the scraper
    buildings_df = scrape_midland_buildings(
        csv_path=csv_path,
        output_path=output_path,
        request_delay=0.5,  # Adjust as needed to avoid rate limiting
        max_retries=3,
        log_level=logging.INFO,
        save_incremental=True,
        incremental_save_frequency=5,
        resume_from_existing=True
    )
    
    print(f"Scraping completed. Found {len(buildings_df)} buildings across all districts and property types.")


2025-04-14 08:12:39,697 - INFO - Successfully loaded 17 districts from data/01_raw/midland_ici_area_code.csv
2025-04-14 08:12:39,698 - INFO - Filtered to 16 districts (excluding 'All Districts')
Scraping progress: 100%|██████████| 48/48 [01:00<00:00,  1.26s/it] , District=Wong Tai Sin District, Type=Shop            

Scraping completed. Found 970 buildings across all districts and property types.


In [17]:
buildings_df.to_csv('midland_buildings.csv', index=False)

In [13]:
import time
import pandas as pd
from tqdm import tqdm
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def initialize_driver():
    """Automatically installs and initializes ChromeDriver in headless mode"""
    chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

def detect_and_replace_svg(icon_url):
    """Convert SVG icons to Yes/No values"""
    if 'PD94bW' in icon_url and "jUwl" in icon_url:
        return "Yes"
    elif 'PD94bW' in icon_url and "gNTAi" in icon_url:
        return "No"
    return "Unknown"

def scrape_building_info(row, driver):
    """Scrape detailed information for individual buildings"""
    try:
        url = row['Detail URL'] + "?lang=english"
        driver.get(url)
        time.sleep(1.5)  # Reduced sleep for faster scraping
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        building_info = {
            'Building Name': row['Building Name'],
            'Address': row['Address'],
            'URL': url
        }
        
        # Extract meta information blocks
        for block in soup.find_all('div', class_='meta-info-container'):
            title = block.find('div', class_='title')
            content = block.find('div', class_='content')
            icon = block.find('div', class_='icon')
            
            if title and (content or icon):
                key = title.text.strip()
                value = content.text.strip() if content else detect_and_replace_svg(icon['src'])
                building_info[key] = value
                
        return building_info
    except Exception as e:
        print(f"Error scraping {row['Building Name']}: {str(e)[:100]}...")
        return None



# Scrape detailed information
print("\nStarting detailed information scraping...")
driver = initialize_driver()
df = pd.read_csv("midlandici_building_list.csv")
results = []

try:
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing buildings"):
        result = scrape_building_info(row, driver)
        if result:
            results.append(result)
            # Progressive save every 5 entries
            if len(results) % 5 == 0:
                pd.DataFrame(results).to_csv("midlandici_detailed_info.csv", index=False)
                
    # Final save
    if results:
        pd.DataFrame(results).to_csv("midlandici_detailed_info.csv", index=False)
        print(f"\nSuccessfully scraped {len(results)}/{len(df)} buildings")

except KeyboardInterrupt:
    print("\nUser interrupted! Saving current progress...")
    pd.DataFrame(results).to_csv("midlandici_detailed_info_PARTIAL.csv", index=False)

finally:
    driver.quit()


,sbu,id,nameEn,nameZh,addressEn,addressZh,__typename,district_id,district_name_en,district_name_cn,property_type,property_type_code,district_property_type
0,mr_ind,B000025468,YIUGA FTY BLDG,耀基工廈,62 VICTORIA ROAD,域多利道62號,Building,1,Central and Western District,中西區,Industrial,mr_ind,Central and Western District_Industrial
1,mr_comm,B000115397,WOHO,WOHO,119 Jervois Street,蘇杭街119號,Building,1,Central and Western District,中西區,Office,mr_comm,Central and Western District_Office
2,mr_comm,B000112477,The Henderson,The Henderson,2 Murray Road,美利道2號,Building,1,Central and Western District,中西區,Office,mr_comm,Central and Western District_Office
3,mr_comm,B000089563,L L Tower,L L Tower,2-4 Shelley Street,些利街2-4號,Building,1,Central and Western District,中西區,Office,mr_comm,Central and Western District_Office
4,mr_comm,B000058939,Two International Finance Centre,國際金融中心二期,8 Finance Street,金融街8號,Building,1,Central and Western District,中西區,Office,mr_comm,Central and Western District_Office
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,mr_comm,B000111508,The Burrow,The Burrow,212 Choi Hung Road,彩虹道212號,Building,8,Wong Tai Sin District,黃大仙區,Office,mr_comm,Wong Tai Sin District_Office
966,mr_comm,B000080050,On Tin Centre,安田中心,1-3 Sheung Hei Street,雙喜街1-3號,Building,8,Wong Tai Sin District,黃大仙區,Office,mr_comm,Wong Tai Sin District_Office
967,mr_comm,B000059457,One Portside,宏基中心一期,29 Tai Yau Street,大有街29號,Building,8,Wong Tai Sin District,黃大仙區,Office,mr_comm,Wong Tai Sin District_Office
968,mr_comm,B000080611,Maxgrand Plaza,萬迪廣場,3 Tai Yau Street,大有街3號,Building,8,Wong Tai Sin District,黃大仙區,Office,mr_comm,Wong Tai Sin District_Office


In [20]:
# Scrape for Building Details

import time
import pandas as pd
from tqdm import tqdm
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import re

def initialize_driver():
    """Automatically installs and initializes ChromeDriver in headless mode"""
    chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

def detect_and_replace_svg(icon_url):
    """Convert SVG icons to Yes/No values"""
    if icon_url and 'PD94bW' in icon_url:
        if "jUwl" in icon_url:
            return "Yes"
        elif "gNTAi" in icon_url:
            return "No"
    return "Unknown"

def clean_for_url(text):
    """Clean text for use in URL"""
    # Replace spaces with hyphens
    text = text.replace(' ', '-')
    # Remove special characters
    text = re.sub(r'[^\w\-]', '', text)
    # Remove consecutive hyphens
    text = re.sub(r'-+', '-', text)
    return text

def construct_url(row):
    """Construct the URL for detailed scraping based on building information"""
    building_type = row['__typename'].lower()
    building_id = row['id']
    building_name = clean_for_url(row['nameEn'])
    return f"https://www.midlandici.com.hk/ics/property/{building_type}/details/{building_id}/{building_name}?lang=english"

def scrape_building_info(row, driver):
    """Scrape detailed information for individual buildings"""
    try:
        url = construct_url(row)
        driver.get(url)
        time.sleep(1.5)  # Sleep for 1.5 seconds to ensure page loads
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        building_info = {
            'id': row['id'],
            'Building Name': row['nameEn'],
            'Building Name (Chinese)': row['nameZh'],
            'Address': row['addressEn'],
            'Address (Chinese)': row['addressZh'],
            'District': row['district_name_en'],
            'Property Type': row['property_type'],
            'URL': url
        }
        
        # Extract meta information blocks
        for block in soup.find_all('div', class_='meta-info-container'):
            title = block.find('div', class_='title')
            content = block.find('div', class_='content')
            icon = block.find('div', class_='icon')
            
            if title and (content or icon):
                key = title.text.strip()
                if content:
                    value = content.text.strip()
                elif icon and 'src' in icon.attrs:
                    value = detect_and_replace_svg(icon['src'])
                else:
                    value = "Unknown"
                building_info[key] = value
                
        return building_info
    except Exception as e:
        print(f"Error scraping {row['nameEn']}: {str(e)[:100]}...")
        return None

def scrape_buildings(input_csv, output_file):
    """
    Scrape detailed information for buildings listed in input_csv and save to output_file.
    Skip buildings that are already in the output file.
    
    Args:
        input_csv (str): Path to the input CSV file
        output_file (str): Path to the output CSV file
    
    Returns:
        pd.DataFrame: DataFrame containing the scraped information
    """
    # Read input CSV
    df_input = pd.read_csv(input_csv)
    
    # Check if output file exists, and read it if it does
    existing_ids = set()
    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        df_output = pd.read_csv(output_file)
        if 'id' in df_output.columns:
            existing_ids = set(df_output['id'])
    
    # Filter out buildings that are already in the output file
    df_to_scrape = df_input[~df_input['id'].isin(existing_ids)]
    
    if len(df_to_scrape) == 0:
        print("All buildings are already scraped. Nothing to do.")
        if os.path.exists(output_file):
            return pd.read_csv(output_file)
        else:
            return pd.DataFrame()
    
    # Initialize Chrome driver
    driver = initialize_driver()
    results = []
    
    try:
        print(f"Scraping detailed information for {len(df_to_scrape)} buildings...")
        for index, row in tqdm(df_to_scrape[:10].iterrows(), total=len(df_to_scrape), desc="Processing buildings"):
            result = scrape_building_info(row, driver)
            if result:
                results.append(result)
        
        # Create DataFrame from results
        df_results = pd.DataFrame(results)
        
        # If output file exists, append new results to it
        if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
            df_existing = pd.read_csv(output_file)
            df_combined = pd.concat([df_existing, df_results], ignore_index=True)
            df_combined.to_csv(output_file, index=False)
            print(f"Added {len(df_results)} new buildings to {output_file}")
            return df_combined
        else:
            # Save results to output file
            df_results.to_csv(output_file, index=False)
            print(f"Saved {len(df_results)} buildings to {output_file}")
            return df_results
    
    finally:
        driver.quit()



In [22]:

# Run the example if script is executed directly
if __name__ == "__main__":
    # Specify your input and output file paths
    input_file = "midland_buildings.csv"
    output_file = "midland_detailed_info.csv"
    
    # Run the scraper
    results_df = scrape_buildings(input_file, output_file)
    
    results_df.head()


2025-04-14 08:28:51,693 - INFO - Chromedriver is already installed.


Scraping detailed information for 960 buildings...


Processing buildings:   1%|          | 10/960 [00:24<38:39,  2.44s/it]

Added 10 new buildings to midland_detailed_info.csv


In [26]:
# Try Get Method

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import time

def clean_for_url(text):
    """Clean text for URL formatting by removing apostrophes and replacing spaces with hyphens."""
    text = text.replace("'", "")  # Remove apostrophes
    return '-'.join(text.split()).strip()

def construct_url(row):
    """Construct the URL for detailed scraping based on building information."""
    return f"https://www.midlandici.com.hk/ics/property/{row['__typename'].lower()}/details/{row['id']}/{clean_for_url(row['nameEn'])}?lang=english"

def scrape_with_requests(row):
    """Scrape building details using requests library."""
    try:
        url = construct_url(row)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        building_info = {
            'id': row['id'],
            'Building Name': row['nameEn'],
            'URL': url
        }

        # Extract meta information
        for block in soup.find_all('div', class_='meta-info-container'):
            title = block.find('div', class_='title')
            content = block.find('div', class_='content')
            
            if title and content:
                key = title.text.strip()
                value = content.text.strip()
                building_info[key] = value
                
        return building_info
    except Exception as e:
        print(f"Request failed for {row['nameEn']}: {str(e)}")
        return None

def process_buildings(input_csv, output_file):
    """
    Process buildings with GET requests and save results to output file.
    
    Args:
        input_csv (str): Path to the input CSV file.
        output_file (str): Path to the output CSV file.
    
    Returns:
        pd.DataFrame: DataFrame containing the scraped information.
    """
    # Read input CSV
    df_input = pd.read_csv(input_csv)
    
    # Check existing data
    existing_ids = set()
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        existing_ids = set(existing_df['id']) if 'id' in existing_df.columns else set()
    
    # Filter out buildings already in the output file
    df_to_scrape = df_input[~df_input['id'].isin(existing_ids)]
    
    if len(df_to_scrape) == 0:
        print("All buildings are already scraped. Nothing to do.")
        if os.path.exists(output_file):
            return pd.read_csv(output_file)
        else:
            return pd.DataFrame()
    
    results = []
    
    # Process buildings with tqdm progress bar and dynamic suffix
    with tqdm(total=len(df_to_scrape), desc="Processing buildings", unit="building") as pbar:
        for _, row in df_to_scrape.iterrows():
            # Update tqdm suffix with building name only
            pbar.set_postfix_str(f"Building: {row['nameEn']}")
            
            result = scrape_with_requests(row)
            if result:
                results.append(result)
            
            # Add a small delay to avoid overwhelming the server
            time.sleep(1)
            
            pbar.update(1)

    # Save results to output file
    if results:
        df_results = pd.DataFrame(results)
        
        # Append new data to existing file or create a new one
        if os.path.exists(output_file):
            df_existing = pd.read_csv(output_file)
            df_combined = pd.concat([df_existing, df_results], ignore_index=True)
            df_combined.to_csv(output_file, index=False)
            print(f"Added {len(df_results)} new records to {output_file}")
            return df_combined
        else:
            df_results.to_csv(output_file, index=False)
            print(f"Saved {len(df_results)} buildings to {output_file}")
            return df_results

# Example usage
if __name__ == "__main__":
    input_csv_path = "midland_buildings.csv"
    output_csv_path = "midland_detailed_info.csv"
    
    result_df = process_buildings(input_csv_path, output_csv_path)
    
    print("\nScraping complete!")



Processing buildings:   8%|▊         | 72/950 [01:39<20:13,  1.38s/building, Building: Effectual Building]                  


KeyboardInterrupt: 